In [9]:
import numpy as np


class LinearRegression(object):
    def __init__(self, fit_intercept=True, copy_X=True):
        self.fit_intercept = fit_intercept
        self.copy_X = copy_X

        self._coef = None
        self._intercept = None
        self._new_X = None

    def fit(self, X, y):
        """
        Linear regression 모델을 적합한다.
        Matrix X와 Vector Y가 입력 값으로 들어오면 Normal equation을 활용하여, weight값을
        찾는다. 이 때, instance가 생성될 때, fit_intercept 설정에 따라 fit 실행이 달라진다.
        fit을 할 때는 입력되는 X의 값은 반드시 새로운 변수(self._new_X)에 저장
        된 후 실행되어야 한다.
        fit_intercept가 True일 경우:
            - Matrix X의 0번째 Column에 값이 1인 column vector를추가한다.
        적합이 종료된 후 각 변수의 계수(coefficient 또는 weight값을 의미)는 self._coef와
        self._intercept_coef에 저장된다. 이때 self._coef는 numpy array을 각 변수항의
        weight값을 저장한 1차원 vector이며, self._intercept_coef는 상수항의 weight를
        저장한 scalar(float) 이다.
        Parameters
        ----------
        X : numpy array, 2차원 matrix 형태로 [n_samples,n_features] 구조를 가진다
        y : numpy array, 1차원 vector 형태로 [n_targets]의 구조를 가진다.
        Returns
        -------
        self : 현재의 인스턴스가 리턴된다
        """
        self._new_X = np.array(X)
        y = y.reshape(-1,1)

        if self.fit_intercept:
            self._new_X = np.concatenate((np.array([1]*X.shape[0]).reshape(-1,1),X),axis=1)
        
        weights = np.linalg.inv(
            self._new_X.T.dot(self._new_X)).dot(self._new_X.T.dot(y))
        
        if self.fit_intercept:
            self._coef = weights[1:]
            self._intercept = weights[0]
        else:
            self._coef = weights[1:]
            

    def predict(self, X):
        """
        적합된 Linear regression 모델을 사용하여 입력된 Matrix X의 예측값을 반환한다.
        이 때, 입력된 Matrix X는 별도의 전처리가 없는 상태로 입력되는 걸로 가정한다.
        fit_intercept가 True일 경우:
            - Matrix X의 0번째 Column에 값이 1인 column vector를추가한다.
        normalize가 True일 경우:
            - Standard normalization으로 Matrix X의 column 0(상수)를 제외한 모든 값을
              정규화을 실행함
            - 정규화를 할때는 self._mu_X와 self._std_X 에 있는 값을 사용한다.
        Parameters
        ----------
        X : numpy array, 2차원 matrix 형태로 [n_samples,n_features] 구조를 가진다
        Returns
        -------
        y : numpy array, 예측된 값을 1차원 vector 형태로 [n_predicted_targets]의
            구조를 가진다.
        """
        test_X = np.array(X)
        
        if self._intercept:
            test_X = np.concatenate((np.array([1]*X.shape[0]).reshape(-1,1),test_X),axis=1)
            
            weights = np.concatenate(([self._intercept], self._coef), axis=0)
        
        else:
            weights = self._coef
        
        return test_X.dot(weights)
            

    @property
    def coef(self):
        return self._coef

    @property
    def intercept(self):
        return self._intercept

In [2]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'x':[77,21,22,20,36],
                   'y':[79.775152,23.177279,25.609262,17.857388,41.849864]})
df

,x,y
0,77,79.775152
1,21,23.177279
2,22,25.609262
3,20,17.857388
4,36,41.849864


In [3]:
X = df["x"].values.reshape(-1,1)
y = df["y"].values

# build model

In [10]:
lr = LinearRegression(fit_intercept=True)

In [11]:
lr.fit(X,y)

In [12]:
lr.intercept

array([1.33731868])

In [13]:
lr.coef

array([[1.03171791]])

In [14]:
lr.predict(X)[:]

array([[80.77959751],
       [23.00339472],
       [24.03511263],
       [21.97167682],
       [38.47916333]])

# VALIDATION

In [15]:
from sklearn import linear_model
sk_lr = linear_model.LinearRegression(normalize=False)
sk_lr.fit(X, y)

LinearRegression()

In [17]:
sk_lr.intercept_

1.3373186796330785

In [18]:
sk_lr.coef_

array([1.03171791])

In [19]:
import numpy.testing as npt
npt.assert_almost_equal(sk_lr.intercept_, lr.intercept)

In [20]:
np.isclose(lr.coef, sk_lr.coef_)

array([[ True]])

In [23]:
df_test = pd.DataFrame({'x':[24,50,15,38,87],
                   'y':[21.5494,47.464463,17.218656,36.586398,87.2889]})
df_test

,x,y
0,24,21.549400
1,50,47.464463
2,15,17.218656
3,38,36.586398
4,87,87.288900


In [24]:
X_test = df["x"].values.reshape(-1,1)

In [25]:
lr.predict(X_test)[:5]

array([[80.77959751],
       [23.00339472],
       [24.03511263],
       [21.97167682],
       [38.47916333]])

In [26]:
sk_lr.predict(X_test)[:5]

array([80.77959751, 23.00339472, 24.03511263, 21.97167682, 38.47916333])

# rescaled

In [27]:
mu_X = np.mean(X, axis=0)
std_X = np.std(X, axis=0)

rescaled_X = (X - mu_X) / std_X

In [ ]:
rescaled_X[:5]

# validation

In [ ]:
lr.fit(rescaled_X, y)

In [ ]:
lr.coef

In [ ]:
lr.intercept

In [ ]:
sk_lr.fit(rescaled_X, y)

In [ ]:
sk_lr.coef_

In [ ]:
sk_lr.intercept_